In [9]:
import os
import requests
import datetime
from zipfile import ZipFile
import ftplib 
import cdsapi
import yaml
import xarray as xr
import glob
import pandas as pd
import gzip
import shutil

In [2]:
#This file contains configuration details like API keys and passwords
# global_vars = yaml.safe_load(open('/home/ds4114/repos/LDEO_Ocean_CO2_Residual/config.yml', 'r') ) #hardcoded
global_vars = yaml.safe_load(open('/home/afay/LDEO_residual/Devan_code/config-amanda.yml', 'r') )
    #previously #global_vars = yaml.safe_load(open('../config.yml', 'r') )

In [3]:
#This has custom functions - specifically the "download" function 
#%run ./00_custom_functions.ipynb
#%run '/home/ds4114/repos/LDEO_Ocean_CO2_Residual/code/00_custom_functions.ipynb'
%run './00_custom_functions.ipynb'

In [4]:
#Data will be saved at this location in folders such as root/SST/originals/ and root/SSS/orginals/
#Note that these folder locations must already be created as a manual validation step
download_folder_root = global_vars['download_folder']
print(download_folder_root)

/data/artemis/observations/online_data_for_reconstructions/


In [5]:
#The following two variables are used to acquire select data when it is uploaded by month or year.
    #Data that is not uploaded by year includes: SST (NOAA); MLD (deBoyer & Argo), fCO2 (SOCAT), xCO2, Coastal, SeaFlux
#These set the start and end years (inclusive) and do not need to be changed.
#Some years/months of data may not available (because prior to when data was gathered or too recent for the source).
    #In those cases any available data is obtained in this range. Specifically, 
        #SST (NOAA) data only 1981-present
        #SST (ERA5) data only 1979-present
        #SST (JRA55) data only 1958-2023
        #SSS data only 1900-present
        #MLD (deBoyer and Argo) data only an averaged 12 months
        #CHL data only 1997-present
        #fCO2 data only 1970-2022
        #SLP data only 1979-2022
        #xCO2 data only 1979-present
        #Coastal data only an averaged 12 months 
        #SeaFlux data only 1982-2022
    #These limitations are hardcoded so other sources or links would be needed to download outside of this range 
acquisition_start_year = 1959 
acquisition_end_year = 2023  

#This variable sets the output filetype for SSS and CHL data and needs to specified explicitly because of the unique way the are downloaded.
#By default, all data source downloads will default to the netcdf format unless using cloud storage. 
#When using cloud storage, it is recommended to use ARCO (Analysis-Ready Cloud-Optimized) formats like Zarr over NetCDF
output_file_type = '.zarr' if download_folder_root[0:5] == 'gs://' else '.nc'

#This variable is used for naming files (SST & xCO2)
today_yearmonth = datetime.datetime.now().strftime('%Y%m')

In [6]:
#Some data (SLP, SST) requires an account on European Centre for Medium-Range Weather Forecasts and an API key
#More info can be found https://cds.climate.copernicus.eu/api-how-to
#Once the packages is installed (conda install -c conda-forge cdsapi), we need to install your API key using code below

cds_url= "url: https://cds.climate.copernicus.eu/api/v2"
cds_key= 'key: '+global_vars['cds_api_key']  #from the configuration file
file = os.path.expanduser('~')+'/.cdsapirc'
if not (os.path.isfile(file)): 
    cds_file = open(file, "w")
    cds_file.write(cds_url+'\n')
    cds_file.write(cds_key)
    cds_file.close()
else:
    print("API key already installed")

API key already installed


## Global Ocean Biogeochemistry Model (GOBM)
#### Global Carbon Budget
This data is not downloaded publically and is recieved via email. The result is below as an example.

In [7]:
#TBD where to download and where to save
xr_open_dataset_custom('/data/artemis/simulations/GCB/2023models/GCB-2023_models_sfco2_simA_1959-2022.nc')

<xarray.Dataset>
Dimensions:         (time: 768, lon: 360, lat: 180)
Coordinates:
  * time            (time) datetime64[ns] 1959-01-15 1959-02-15 ... 2022-12-15
  * lon             (lon) float64 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
  * lat             (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Data variables:
    CNRM-ESM2-1     (time, lat, lon) float64 ...
    FESOM2-REcoM    (time, lat, lon) float64 ...
    IPSL            (time, lat, lon) float64 ...
    MOM6-Princeton  (time, lat, lon) float64 ...
    MRI-ESM2-2      (time, lat, lon) float64 ...
    NorESM-OC1.2    (time, lat, lon) float64 ...
    CESM-ETHZ       (time, lat, lon) float64 ...
    ACCESS          (time, lat, lon) float64 ...

## Network Mask

#### Land Sea Mask

In [8]:
#need to load some datasets for filtering ocean vs land. Add all base data here and in 02 processing, filter out artic and others

#NCEP land-sea mask (https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.surface.html)
land_mask_direct_url = 'https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis/surface/land.nc'
land_mask_destination_folder = download_folder_root+r'Masks/originals/'
land_mask_destination_filename = 'mask_NOAA_land-sea_20220916.nc'
download(land_mask_direct_url, land_mask_destination_folder, land_mask_destination_filename, create_dest=True) 
#here 1 means land and 0 is ocean. We flip in the 02 processing code

File mask_NOAA_land-sea_20220916.nc already exists at /data/artemis/observations/online_data_for_reconstructions/Masks/originals/ - (skipping download from https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis/surface/land.nc )


In [9]:
#Not sure if this is the right source but data from here https://www.gebco.net/data_and_products/gridded_bathymetry_data/. 
#And historical https://www.gebco.net/data_and_products/historical_data_sets/#gebco_one
#original version used: xr_open_dataset_custom('/local/data/artemis/observations/GEBCO_2014/processed/GEBCO_2014_1x1_global.nc') but nothing in the originals folder so not sure source

#Data comes as a zip which we will extract into netcdf. 
#This file is large (4GB compresesd, 7 GB uncompressed) meaning it may not be possible to do with temporary storage limitations on Pangeo
#the 2023 file is large, hard to download, and cannot be regrid with xesmf (buffer size error) so we should go with smaller one
if output_file_type == '.zarr':
    print("ERROR - Space limitations will occur when downloading and converting to .zarr. Try manually downlaoding offline first")
else:
    #topography_direct_url = 'https://www.bodc.ac.uk/data/open_download/gebco/gebco_2023/zip/'
    #topography_direct_url = 'https://www.bodc.ac.uk/data/open_download/gebco/GEBCO_1MIN/zip/'
    topography_direct_url = 'https://www.bodc.ac.uk/data/open_download/gebco/GEBCO_30SEC/zip/'
    topography_destination_folder = download_folder_root+r'Masks/originals/'
    #topography_destination_filename = 'mask_GEBCO_topography-2023-15arc.zip'
    #topography_destination_filename = 'mask_GEBCO_topography-2008-60arc.zip'
    topography_destination_filename = 'mask_GEBCO_topography-2014-30arc.zip'
    download(topography_direct_url, topography_destination_folder, topography_destination_filename) 

    #extract files
    ZipFile(topography_destination_folder+topography_destination_filename).extractall(topography_destination_folder)
    #could do some clean up or renaming here if needed. Some pdfs are included in the zips

File mask_GEBCO_topography-2014-30arc.zip already exists at /data/artemis/observations/online_data_for_reconstructions/Masks/originals/ - (skipping download from https://www.bodc.ac.uk/data/open_download/gebco/GEBCO_30SEC/zip/ )


In [16]:
#xr_open_dataset_custom('/local/data/artemis/observations/GEBCO_2014/processed/GEBCO_2014_1x1_global.nc') #topography
#xr_open_dataset_custom('/data/artemis/workspace/ds4114/online_data/Masks/originals/mask_GEBCO_topography-2023-15arc.nc') #topography
#xr_open_dataset_custom('/data/artemis/workspace/ds4114/online_data/Masks/originals/GEBCO_2023.nc') #topography

### SOCAT fCO2

In [17]:

#More info can be found at https://www.socat.info/index.php/data-access/
#want to go down to gridded files and copy the linked address for global under the Main Source (NCEI) path

fco2_direct_url = 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0293257/SOCATv2024_Gridded_Dat/SOCATv2024_tracks_gridded_monthly.nc' #thru 2023-12
# fco2_direct_url = 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0278913/SOCATv2023_Gridded_Dat/SOCATv2023_tracks_gridded_monthly.nc'  #thru 2022-12
      #previously 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0253659/SOCATv2022_Gridded_Dat/SOCATv2022_tracks_gridded_monthly.nc'  #thru 2021-12
fco2_destination_folder = download_folder_root+r'pCO2/originals/'    
fco2_destination_filename = 'fCO2_SOCOVV_SOCAT-gridded-monthly_2024.nc'    #filename year changed from source to denote data date, not publish date
download(fco2_direct_url, fco2_destination_folder, fco2_destination_filename)

Saving https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0293257/SOCATv2024_Gridded_Dat/SOCATv2024_tracks_gridded_monthly.nc to /data/artemis/observations/online_data_for_reconstructions/pCO2/originals/fCO2_SOCOVV_SOCAT-gridded-monthly_2024.nc...
Complete


# Varaibles needed for ML
(below there is a section for variables needed only for the flux calculation)

## xCO2 Mauna Loa and Global
#### (NOAA)

xCO2 comes from NOAA. More information: https://gml.noaa.gov/ccgg/trends/gl_data.html and https://gml.noaa.gov/ccgg/mbl/data.php

We need global xCO2 for observation years for the Machine Learning (1982-present). This is what we use the global file for.

We ALSO need atmospheric fCO2, with variable lat/lon, going back to 1959 for flux calculations. This is what we use the MBL file for. This will be done in the 02_data_processing workbook, but we use the Mauna Loa time series to go back in time.


So we need 3 things to download here:
1 global monthly mean csv (https://gml.noaa.gov/aftp/products/trends/co2/co2_mm_gl.csv #monthly global mean values. this is pretty quickly updated (currently goes through 5/2024 and it's only 8/24 right now)

2 mauna loa mean csv (https://gml.noaa.gov/aftp/products/trends/co2/co2_mm_mlo.csv #monthly mauna loa values)
3 surface marine boundary layer lat varying file

The NOAA surface mbl is updated each summer. I worked with Xin Lan (xin.lan@noaa.gov) to get an advanced version. I wrote the code here to download from the internet, but for the advanced version I got I put it on artemis (/local/data/artemis/observations/globalview/originals/) and will use that for now.
This is all from this website: https://gml.noaa.gov/ccgg/mbl/data.php and select "Surface" option and download data.



### Global monthly mean xco2
(needed for Machine Learning)

In [6]:

xco2_direct_url = "https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_gl.csv"
xco2_destination_folder = download_folder_root+r'xCO2/originals'
today_yearmonth = datetime.datetime.now().strftime('%Y%m')
xco2_destination_filename = 'xCO2_NOAA_xCO2-mm-gl-monthly_197901-'+today_yearmonth+'.csv'
download(xco2_direct_url, xco2_destination_folder, xco2_destination_filename)

Saving https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_gl.csv to /data/artemis/observations/online_data_for_reconstructions/xCO2/originals/xCO2_NOAA_xCO2-mm-gl-monthly_197901-202408.csv...
Complete


### Mauna Loa monthly mean xco2
(needed for flux calculation)

In [7]:
#But this is fine for the ML method input!
xco2_direct_url = "https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_mlo.csv"
xco2_destination_folder = download_folder_root+r'xCO2/originals'
today_yearmonth = datetime.datetime.now().strftime('%Y%m')
xco2_destination_filename = 'xCO2_NOAA_xCO2-mm-mlo-monthly_197901-'+today_yearmonth+'.csv'
download(xco2_direct_url, xco2_destination_folder, xco2_destination_filename,overwrite=False)

Saving https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_mlo.csv to /data/artemis/observations/online_data_for_reconstructions/xCO2/originals/xCO2_NOAA_xCO2-mm-mlo-monthly_197901-202409.csv...
Complete


### surface marine boundary layer xco2 
(needed for flux calculation)
Just save it as a txt file here and process it in the 02 workbook.


In [51]:
#This is an older version of the surface mbl thats on the noaa website. hasnt officially been updated yet.
#I manually added a file in that I got from Xin at NOAA to get the 2023 values
xco2_surface_mbl_direct_url = "https://gml.noaa.gov/ccgg/mbl/tmp/co2_GHGreference.772098448_surface.txt"
xco2_surface_mbl_destination_folder = download_folder_root+r'xCO2/originals/'
lastmonth_yearmonth = (datetime.date.today().replace(day=1) - datetime.timedelta(days=1)).strftime('%Y%m') #assuming data is updated
xco2_surface_mbl_destination_filename = 'xCO2_NOAA_xCO2-mbl-monthly_197901_downloaded_on'+lastmonth_yearmonth+'.txt'
download(xco2_surface_mbl_direct_url, xco2_surface_mbl_destination_folder, xco2_surface_mbl_destination_filename, create_dest=True) 

Saving https://gml.noaa.gov/ccgg/mbl/tmp/co2_GHGreference.772098448_surface.txt to /data/artemis/observations/online_data_for_reconstructions/xCO2/originals/xCO2_NOAA_xCO2-mbl-monthly_197901_downloaded_on202407.txt...
Complete


## Sea Surface Temperature


### NOAA

In [10]:
#NOAA SST is easy because NOAA uses the same URL for all data across time periods (1981-09 to present only)
#More info found on https://psl.noaa.gov/data/gridded/data.noaa.oisst.v2.highres.html
sst_direct_url = 'https://downloads.psl.noaa.gov/Datasets/noaa.oisst.v2.highres/sst.mon.mean.nc' 
sst_destination_folder = download_folder_root+r'SST/originals/'
sst_destination_filename = 'SST_NOAA_OI-V2-HighRes_198109-'+today_yearmonth+'.nc'  #data is only from 1981
download(sst_direct_url, sst_destination_folder, sst_destination_filename)

Saving https://downloads.psl.noaa.gov/Datasets/noaa.oisst.v2.highres/sst.mon.mean.nc to /data/artemis/observations/online_data_for_reconstructions/SST/originals/SST_NOAA_OI-V2-HighRes_198109-202408.nc...
Complete


## Salinity SSS

A copy of code with larger timeframe from the Residual method. We dont need to actually download because we have it already but code is for reference

In [9]:
#SST requires downloading yearly zip files and extracting each month from each  (1900-01 to present only)
#More info found on https://www.metoffice.gov.uk/hadobs/en4/download-en4-2-2.html
#Note: because the file names in the zip are not known until extracting, this cell will execute even if the data already has been downloaded

sss_direct_url_base = 'https://www.metoffice.gov.uk/hadobs/en4/data/en4-2-1/EN.4.2.2/'
sss_destination_filename_base = 'SSS_Met-Office-Hadley-Centre_EN422f-g10-analyses_' #_197901+
sss_destination_folder = download_folder_root+r'SSS/originals/'
for year in range(acquisition_start_year, acquisition_end_year+1, 1):
    if year >= 2021: sss_direct_url_base = 'https://www.metoffice.gov.uk/hadobs/en4/data/en4-2-1/' #2021 and onward the URL changes, though both URL zips exists. Not clear why.
    url_file = 'EN.4.2.2.analyses.g10.'+str(year)+'.zip'
    sss_direct_url = sss_direct_url_base + url_file
    sss_destination_filename = sss_destination_filename_base+str(year)+'.zip'
    
    #download to tmp, extract a year, then transfer all to destination
    tmp_folder = '../tmp/'
    download_to_folder(sss_direct_url, tmp_folder, sss_destination_filename,overwrite=False, create_dest=True)
    ZipFile(tmp_folder+sss_destination_filename).extractall(tmp_folder)
    files = glob.glob(tmp_folder+'EN.4.2.2.f.analysis.g10.*.nc')
    for f in files:  #for each file extracted
        basename = os.path.basename(f)
        tmp_xr = xr.open_dataset(tmp_folder + basename)
        output_xarray_with_date(tmp_xr, sss_destination_folder, basename, filetype=output_file_type, with_date=False) 
            #Note the filename is kept as the original from the .zip here to be compatible with other work. To output the file with a different name, edit the dest_file string parameter.
        os.remove(os.path.join(tmp_folder,f)) #remove tmp file
    
    #remove zip file too
    os.remove(os.path.join(tmp_folder,sss_destination_filename)) #remove tmp file
print("SSS Complete")

Saving https://www.metoffice.gov.uk/hadobs/en4/data/en4-2-1/EN.4.2.2/EN.4.2.2.analyses.g10.1959.zip to ../tmp/SSS_Met-Office-Hadley-Centre_EN422f-g10-analyses_1959.zip...
Complete
Saved EN.4.2.2.f.analysis.g10.195901.nc to /data/artemis/observations/online_data_for_reconstructions/SSS/originals/
Saved EN.4.2.2.f.analysis.g10.195902.nc to /data/artemis/observations/online_data_for_reconstructions/SSS/originals/
Saved EN.4.2.2.f.analysis.g10.195903.nc to /data/artemis/observations/online_data_for_reconstructions/SSS/originals/
Saved EN.4.2.2.f.analysis.g10.195904.nc to /data/artemis/observations/online_data_for_reconstructions/SSS/originals/
Saved EN.4.2.2.f.analysis.g10.195905.nc to /data/artemis/observations/online_data_for_reconstructions/SSS/originals/
Saved EN.4.2.2.f.analysis.g10.195906.nc to /data/artemis/observations/online_data_for_reconstructions/SSS/originals/
Saved EN.4.2.2.f.analysis.g10.195907.nc to /data/artemis/observations/online_data_for_reconstructions/SSS/originals/
S

## Mixed Layer Depth (MLD)
two options for this. typically weh ave used deBoyer but maybe update with Argo version now?


#### deBoyer

In [18]:
#MLD data is just one year of data that was processed by the author using several years of temperature profiles (data from 1941-2008, partially missing)
#More info found on https://cerweb.ifremer.fr/deboyer/mld/Surface_Mixed_Layer_Depth.php

mld_direct_url = 'https://cerweb.ifremer.fr/deboyer/data/mld_DT02_c1m_reg2.0.nc'
mld_destination_folder = download_folder_root+r'MLD/originals/'
mld_destination_filename = 'MLD_IFREMER-deBoyer_DT02-c1m_2008.nc'   #2008 though data is blended across years 
download(mld_direct_url, mld_destination_folder, mld_destination_filename, create_dest=True)

Saving https://cerweb.ifremer.fr/deboyer/data/mld_DT02_c1m_reg2.0.nc to /data/artemis/observations/online_data_for_reconstructions/MLD/originals/MLD_IFREMER-deBoyer_DT02-c1m_2008.nc...
Complete


#### Argo

In [19]:
# This MLD climatology source is from UC San Diego who used Argo data to create the climatology
# More info can be found on http://mixedlayer.ucsd.edu/ and https://www.seanoe.org/data/00311/42182/#56126
mld_direct_url = 'http://mixedlayer.ucsd.edu/data/Argo_mixedlayers_monthlyclim_04142022.nc'
mld_destination_folder = download_folder_root+r'MLD/originals/'
mld_destination_filename = 'MLD_UCSD-Argo_mixedlayers-monthlyclim_2022.nc'   #2022 though data is blended across years 
download(mld_direct_url, mld_destination_folder, mld_destination_filename, create_dest=True)

Saving http://mixedlayer.ucsd.edu/data/Argo_mixedlayers_monthlyclim_04142022.nc to /data/artemis/observations/online_data_for_reconstructions/MLD/originals/MLD_UCSD-Argo_mixedlayers-monthlyclim_2022.nc...
Complete


## Chlorophyll (CHL)

In [20]:
#CHL data needs to be downloaded via FTP in monthly files (1997-09 to present only)
#According to https://www.globcolour.info/products_description_mermet.html, we want to use GSM data only
#Note that the product changes over the years and the file names can vary. However there is a common pattern in naming used below.
#More info found on https://hermes.acri.fr/index.php?class=archive

#FTP requires an account and login:
chl_direct_ftp = 'ftp.hermes.acri.fr'
chl_destination_folder = download_folder_root+r'CHL/originals/'
chl_acquisition_start_year = acquisition_start_year if acquisition_start_year >= 1997 else 1997 #earliest we have is 1997-09 so set it here to prevent errors getting nonexistant data
usr = global_vars['chl_user'] 
psw = global_vars['chl_psw'] 

ftp_server = ftplib.FTP(chl_direct_ftp)
ftp_server.set_pasv(True)
ftp_server.login(usr, psw)

'230 User ftp_gc_VBennington logged in'

In [21]:
#Note the nlst command used below may throw a [WinError 10060] message after logging in. This may be due to how the FTP server is configured or how the local computer is configured. 
#If the following command does not work, there is a broader issue with the connection.
ftp_server.dir()

drwxrwsr-x   3 10048    961403205        0 Aug 20  2021 animation
drwxrwsr-x  11 10048    961403205        0 May 15 07:50 ATLNW
drwxrwsr-x  12 10048    961403205        0 Sep 30  2022 EURO
drwxrwsr-x  13 10048    961403205        0 Jan 30  2023 GLOB
drwxrwsr-x   5 10048    961403205        0 Mar 27  2021 GLOBCOAST
drwxrwsr-x   5 10048    961403205        0 Dec  2  2014 OSS2015


In [22]:
for year in range(chl_acquisition_start_year, acquisition_end_year+1, 1):
    chl_destination_filename = ''
    months = [i for i in range(1,12+1)]
     #if a partial year, reduce available months
    if year == datetime.datetime.now().year: months = [i for i in range(1,datetime.datetime.now().month)]
    elif year == 1997: months = [i for i in range(9,12+1)] #data only available part of 1997
    for month in months:
        chl_destination_filename = 'CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_'+str(year)+str(month).zfill(2)+'.nc'
        ftp_server.cwd(f'/GLOB/merged/month/{year}/{str(month).zfill(2)}/01') #ex. '/GLOB/merged/month/2022/02/01' for Feb 2022
        f = ftp_server.nlst('L3m*GLOB*100*GSM*CHL1*.nc')[0]    #Global view, 100 km resolution, GSM product, CHL1 data
        
        #download to tmp folder first, then check and transfer to destination
        tmp_folder = '../tmp/'
        if not os.path.exists(tmp_folder): os.makedirs(tmp_folder)
        print(f'Downloading {f} as {chl_destination_filename}')
        ftp_server.retrbinary("RETR " + f, open(tmp_folder + chl_destination_filename, 'wb').write)
        tmp_xr = xr.open_dataset(tmp_folder + chl_destination_filename)
        output_xarray_with_date(tmp_xr, chl_destination_folder, chl_destination_filename, filetype=output_file_type, with_date=False)
        del tmp_xr
        os.remove(os.path.join(tmp_folder , chl_destination_filename)) #remove tmp file
            
print("CHL Complete")

Saved CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_199709.nc to /data/artemis/observations/online_data_for_reconstructions/CHL/originals/
Saved CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_199710.nc to /data/artemis/observations/online_data_for_reconstructions/CHL/originals/
Saved CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_199711.nc to /data/artemis/observations/online_data_for_reconstructions/CHL/originals/
Saved CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_199712.nc to /data/artemis/observations/online_data_for_reconstructions/CHL/originals/
Saved CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_199801.nc to /data/artemis/observations/online_data_for_reconstructions/CHL/originals/
Saved CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_199802.nc to /data/artemis/observations/online_data_for_reconstructions/CHL/originals/
Saved CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_199803.nc to /data/artemis/observations/online_data_for_reconstructions/CHL/ori

In [23]:
ftp_server.quit()

'221 Goodbye.'

## pCO2 
### SOCAT fCO2

In [24]:
#pCO2 data is calculated from fCO2 and SST (SOCOVV) and sea level pressure (ECMRWF) so we need to obtain a dataset for both

#fCO2 and SST comes from the Surface Ocean CO2 Variability and Vulnerability group (1970-01 to end of last year only)
#We want to use the same SST data points related to fCO2 from the same source (not using the separate NOAA SST here).
#More info can be found at https://www.socat.info/index.php/data-access/
fco2_direct_url = 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0293257/SOCATv2024_Gridded_Dat/SOCATv2024_tracks_gridded_monthly.nc'  #thru 2022-12
    #previously 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0278913/SOCATv2023_Gridded_Dat/SOCATv2023_tracks_gridded_monthly.nc'  #thru 2022-12  
    #previously 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0253659/SOCATv2022_Gridded_Dat/SOCATv2022_tracks_gridded_monthly.nc'  #thru 2021-12
fco2_destination_folder = download_folder_root+r'pCO2/originals/'    
fco2_destination_filename = 'fCO2_SOCOVV_SOCAT-gridded-monthly_2022.nc'    #filename year changed from source to denote data date, not publish date
download(fco2_direct_url, fco2_destination_folder, fco2_destination_filename, create_dest=True)

Saving https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0293257/SOCATv2024_Gridded_Dat/SOCATv2024_tracks_gridded_monthly.nc to /data/artemis/observations/online_data_for_reconstructions/pCO2/originals/fCO2_SOCOVV_SOCAT-gridded-monthly_2022.nc...
Complete


### Coastal climatology from landschutzer et al 2020 needed or coastal filling method

In [6]:
#Note that this doesnt get updates each year so no need to update in our code

#More info can be found at https://www.ncei.noaa.gov/access/ocean-carbon-acidification-data-system/oceans/MPI-ULB-SOM_FFN_clim.html

coast_direct_url = 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0209633/MPI-ULB-SOM_FFN_clim.nc' 

coast_destination_folder = download_folder_root+r'pCO2/originals/'    
coast_destination_filename = 'coastal_climatology_Landschutzer2020.nc'    #filename year changed from source to denote data date, not publish date
download(coast_direct_url, coast_destination_folder, coast_destination_filename, create_dest=True, overwrite=False)

Saving https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0209633/MPI-ULB-SOM_FFN_clim.nc to /data/artemis/observations/online_data_for_reconstructions/pCO2/originals/coastal_climatology_Landschutzer2020.nc...
Complete


# Extra variables needed only for flux calculation

## Wind
#### ERA5 Wind (WS10 and WS_STD)
#### Only needed for flux calc, not for ML method

for winds, we need to download the hourly winds from ERA5, and then calculate monthly winds speed squared and second moment.
Here is the code to download all the hourly winds and save them. 
When running updates, only do the new year that you are updating.

In [26]:
#example
# xr.open_dataset('/data/artemis/observations/ERA5/Winds_2020-01.nc')
#retrieve: reanalysis-era5-single-levels. variables: '10m_u_component_of_wind', '10m_v_component_of_wind'. --hourly so big...


In [9]:
# variable = ['10m_u_component_of_wind', '10m_v_component_of_wind']
# c = cdsapi.Client()
# c.retrieve(
#     'reanalysis-era5-single-levels-monthly-means',
#     {
#         'format': 'netcdf',
#         'year': year,
#         'variable': variable,
#         'product_type': 'monthly_averaged_reanalysis',
#         'month': months,
#         'time': '00:00'
#     },
#     '../tmp')

In [7]:
%run './00_custom_functions.ipynb'

In [ ]:
#set up to just aquire the last year to update other files, but if you are starting from scratch you need to get all years

In [10]:
variable_list = ['10m_u_component_of_wind', '10m_v_component_of_wind']
acquisition_start_year = 2000 #change here if needed
acquisition_end_year = 2000

wind_destination_folder = download_folder_root+r'Wind/originals/'
wind_destination_filename = ''

for year in range(acquisition_start_year, acquisition_end_year+1, 1):
    
    
    for months in ['04','05','06','07','08','09','10','11','12']: #'01','02','03',
        print(year)
        print("month = ",months)

    # months = []
        if year == datetime.datetime.now().year:   #if a partial year; this prevents erorrs trying to use the API to get future/non-existant data    
            # months = [i for i in range(1,datetime.datetime.now().month -1)]  
            wind_destination_filename = f'Wind_ECMWF_ERA5-10m-uv-wind_{year}-{months}.nc' #'Wind_ERA5-10m-uv-wind_'+str(year)+str(months)+'.nc'  #+str(months)
            #Note, partial years will not automatically be overwritten so you must manually clean up unneeded files
        else:
            wind_destination_filename = f'Wind_ECMWF_ERA5-10m-uv-wind_{year}-{months}.nc' #'Wind_ERA5-10m-uv-wind_'+str(year)+str(months)+'.nc' #'Wind_ECMWF_ERA5-10m-uv-wind_{year}-{month}.nc'
            # months = [i for i in range(1,12+1)]
            # months = [i for i in range(1,4)] #try with just a few months to see how big the file is
    
        cdsapi_custom_download_hourly(year, months, variable_list, wind_destination_folder, wind_destination_filename, overwrite=False, create_dest=True)
    #cdsapi_custom_download(year, months, variable_list, wind_destination_folder, wind_destination_filename, overwrite=False, create_dest=False)
        ct = datetime.datetime.now()
        print("current time:-", ct)
        
print('Complete')

2000
month =  04


2024-09-10 10:57:23,344 INFO Welcome to the CDS.
As per our announcements on the Forum, this instance of CDS will be decommissioned on 26 September 2024 and will no longer be accessible from this date onwards.
Please update your cdsapi package to a version >=0.7.2, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
2024-09-10 10:57:23,345 WARNING MOVE TO CDS-Beta
2024-09-10 10:57:23,346 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-09-10 10:57:23,475 INFO Request is queued

KeyboardInterrupt



In [ ]:
acquisition_end_year

In [ ]:
ct = datetime.datetime.now()
print("current time:-", ct)

## ERA5 SST and pressure
Used for pCO2 atm calculations to go back in time

In [7]:
#This SST dataset comes from European Centre for Medium-Range Weather Forecasts (1979-01 to present only). It is the same as the one used for SLP below.
#More info can be found on https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form
#We installed the API and have an account, so now we can download the ERA5 data via a loop (yearly)
#While this data can be downloaded together with SLP, we have chosen to break up the downloads for clarity and organization

sst_destination_folder = download_folder_root+r'SST/originals/'
for year in range(acquisition_start_year, acquisition_end_year+1, 1):
    sst_destination_filename = ''
    months = []
    if year == datetime.datetime.now().year:   #if a partial year; this prevents erorrs trying to use the API to get future/non-existant data    
        months = [i for i in range(1,datetime.datetime.now().month -1)]  
        sst_destination_filename = 'SST_ECMWF_ERA5-monthly-reanalysis-SST_'+str(year)+'-partial.nc'  
        #Note, partial years will not automatically be overwritten so you must manually clean up unneeded files
    else:
        sst_destination_filename = 'SST_ECMWF_ERA5-monthly-reanalysis-SST_'+str(year)+'.nc'
        months = [i for i in range(1,12+1)]

    if not os.path.isfile(sst_destination_folder + sst_destination_filename):  #if not already downloaded
        c = cdsapi.Client()
        c.retrieve(
            'reanalysis-era5-single-levels-monthly-means',
            {
                'format': 'netcdf',
                'year': year,
                'variable': 'sea_surface_temperature',
                'product_type': 'monthly_averaged_reanalysis',
                'month': months,
                'time': '00:00'
            },
            sst_destination_folder + sst_destination_filename)
    else:
        print(f"File already exists - (skipping download for {year})")
print("Complete")

File already exists - (skipping download for 1959)
File already exists - (skipping download for 1960)
File already exists - (skipping download for 1961)
File already exists - (skipping download for 1962)
File already exists - (skipping download for 1963)
File already exists - (skipping download for 1964)
File already exists - (skipping download for 1965)
File already exists - (skipping download for 1966)
File already exists - (skipping download for 1967)
File already exists - (skipping download for 1968)
File already exists - (skipping download for 1969)
File already exists - (skipping download for 1970)
File already exists - (skipping download for 1971)
File already exists - (skipping download for 1972)
File already exists - (skipping download for 1973)
File already exists - (skipping download for 1974)
File already exists - (skipping download for 1975)
File already exists - (skipping download for 1976)
File already exists - (skipping download for 1977)
File already exists - (skipping

2024-08-30 14:12:36,378 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-30 14:12:36,380 WARNING MOVE TO CDS-Beta
2024-08-30 14:12:36,381 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-08-30 14:12:36,520 INFO Request is queued
2024-08-30 14:12:45,066 INFO Request is running
2024-08-30 14:12:50,234 INFO Request is completed
2024-08-30 14:12:50,236 INFO Downloading https://download-0010-clone.copernicus-climate.eu/cache-compute-0010/cache/data0/adaptor.mars.internal-1725041564.9693885-31528-17-0d

Complete


## Sea Level Pressure
#### MSLP ECMRWF
A copy of code with larger timeframe from the Residual method. We do probably need to download it/update it.

In [11]:
#Sea level pressure comes from European Centre for Medium-Range Weather Forecasts (1940-01 to present only). It is the same source the one used for one SST above.
#More info can be found on https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form
#We installed the API and have an account, so now we can download the ERA5 data via a loop (yearly)
slp_destination_folder = download_folder_root+r'SLP/originals/'    
for year in range(acquisition_start_year, acquisition_end_year+1, 1):
    slp_destination_filename = ''
    months = []
    if year == datetime.datetime.now().year:   #if a partial year; this prevents erorrs trying to use the API to get future/non-existant data    
        months = [i for i in range(1,datetime.datetime.now().month)]  
        slp_destination_filename = 'SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_'+str(year)+'.nc'  #rename if desired for partial year
        #To consider, do we want to automatically overwrite parital years?
    else:
        slp_destination_filename = 'SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_'+str(year)+'.nc'
        months = [i for i in range(1,12+1)]

    cdsapi_custom_download(year, months, 'mean_sea_level_pressure', slp_destination_folder, slp_destination_filename, overwrite=False, create_dest=True)
    
print("SLP Complete")

2024-08-30 13:05:22,343 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-30 13:05:22,345 WARNING MOVE TO CDS-Beta
2024-08-30 13:05:22,346 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-08-30 13:05:22,467 INFO Request is queued
2024-08-30 13:05:23,571 INFO Request is running
2024-08-30 13:05:27,526 INFO Request is completed
2024-08-30 13:05:27,528 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data2/adaptor.mars.internal-1725037526.217433-22902-1-9da5

Transferring to Destination /data/artemis/observations/online_data_for_reconstructions/SLP/originals/SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1959.nc...


2024-08-30 13:05:30,192 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-30 13:05:30,193 WARNING MOVE TO CDS-Beta
2024-08-30 13:05:30,194 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-08-30 13:05:30,336 INFO Request is queued
2024-08-30 13:05:33,064 INFO Request is running
2024-08-30 13:05:38,900 INFO Request is completed
2024-08-30 13:05:38,902 INFO Downloading https://download-0010-clone.copernicus-climate.eu/cache-compute-0010/cache/data9/adaptor.mars.internal-1725037535.0310667-11984-3-fc6

Transferring to Destination /data/artemis/observations/online_data_for_reconstructions/SLP/originals/SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1960.nc...


2024-08-30 13:05:41,463 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-30 13:05:41,464 WARNING MOVE TO CDS-Beta
2024-08-30 13:05:41,465 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-08-30 13:05:41,573 INFO Request is queued
2024-08-30 13:05:42,675 INFO Request is running
2024-08-30 13:05:46,628 INFO Request is completed
2024-08-30 13:05:46,629 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data2/adaptor.mars.internal-1725037544.6229265-16730-18-c0

...

Transferring to Destination /data/artemis/observations/online_data_for_reconstructions/SLP/originals/SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1976.nc...


2024-08-30 13:11:32,997 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-30 13:11:32,998 WARNING MOVE TO CDS-Beta
2024-08-30 13:11:32,998 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-08-30 13:11:33,103 INFO Request is queued
2024-08-30 13:11:34,204 INFO Request is running
2024-08-30 13:11:38,152 INFO Request is completed
2024-08-30 13:11:38,153 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data6/adaptor.mars.internal-1725037896.018212-9515-17-05e1

Transferring to Destination /data/artemis/observations/online_data_for_reconstructions/SLP/originals/SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1977.nc...


2024-08-30 13:11:41,121 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-30 13:11:41,122 WARNING MOVE TO CDS-Beta
2024-08-30 13:11:41,123 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-08-30 13:11:41,273 INFO Request is queued
2024-08-30 13:11:42,373 INFO Request is running
2024-08-30 13:11:46,320 INFO Request is completed
2024-08-30 13:11:46,321 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data2/adaptor.mars.internal-1725037903.5477746-32385-4-385

Transferring to Destination /data/artemis/observations/online_data_for_reconstructions/SLP/originals/SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1978.nc...
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1979.nc already exists - (skipping download from 1979 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1980.nc already exists - (skipping download from 1980 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1981.nc already exists - (skipping download from 1981 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1982.nc already exists - (skipping download from 1982 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1983.nc already exists - (skipping download from 1983 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1984.nc already exists - (skipping download from 1984 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1985.nc already exists - (skipping download from 1985 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1986.nc already exists - (skipping download from 1986 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1987.

## Ice
#### Hadley Office

In [8]:
# Ice from Hadley (starts in Jan 1870 and can get updated versions at https://www.metoffice.gov.uk/hadobs/hadisst/data/download.html to extend)

tmp_folder = '../tmp/'
download(r'https://www.metoffice.gov.uk/hadobs/hadisst/data/HadISST_ice.nc.gz', 
          os.path.join(download_folder_root,'Ice/originals/'),   #better to download to tmp and move
         # tmp_folder,
         'HadISST_ice.nc.gz', create_dest=True) 

File HadISST_ice.nc.gz already exists at /data/artemis/observations/online_data_for_reconstructions/Ice/originals/ - (skipping download from https://www.metoffice.gov.uk/hadobs/hadisst/data/HadISST_ice.nc.gz )


In [9]:
with gzip.open(os.path.join(tmp_folder,'HadISST_ice.nc.gz'), 'rb') as f_in:
        ice_xr = xr.open_dataset(f_in)
output_xarray_with_date(ice_xr, os.path.join(download_folder_root,'Ice/originals/'), 'Ice_Met-Office-Hadley-Centre_HadISST', filetype=output_file_type)
#os.remove(os.path.join(tmp_folder,'HadISST_ice.nc.gz')) #remove tmp file

Saved Ice_Met-Office-Hadley-Centre_HadISST_187001-202405.nc to /data/artemis/observations/online_data_for_reconstructions/Ice/originals/


In [26]:
# ice_xr